How do we get the amplitude of each event? 

We know when they start and how long they are. So we can go back to original data and get the max value within each one.

This is potentially very slow. 


In [1]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs


In [2]:
stem = "/project/amp/brianpm/TemperatureExtremes/Derived/"
obsfil = "CPC_tmax_90pct_event_attributes_c20191118.nc"
mdlfil = "f.e13.FAMIPC5CN.ne30_ne30.beta17.TREFMXAV.90pct_event_attributes_c20191118.nc"

In [3]:
ds = xr.open_dataset(stem+obsfil)
mdl_ds = xr.open_dataset(stem+mdlfil)

In [6]:
# the original data is much bigger:
mdl_f = xr.open_dataset(stem+"../Regridded/f.e13.FAMIPC5CN.ne30_ne30.beta17.t3.cam.h1.TREFMXAV.19650101-20051231.regrid.nc")

In [72]:
quants = xr.open_dataset("/project/amp/brianpm/TemperatureExtremes/Derived/f.e13.FAMIPC5CN.ne30_ne30.beta17.TREFMXAV.dayofyear_quantiles_15daywindow_c20190626.nc")
quants['time'] = np.linspace(1, 365, 365, dtype=int)
medians = quants.sel(quantile=0.5).rename({'time':'dayofyear'})
medians['dayofyear']

<xarray.DataArray 'dayofyear' (dayofyear: 365)>
array([  1,   2,   3, ..., 363, 364, 365])
Coordinates:
    quantile   float64 0.5
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365

In [73]:
# This turns the daily TREFMX value into anomalies w.r.t. the median of the 15-day-window centered on that day
TANOM = mdl_f['TREFMXAV'].groupby("time.dayofyear") - medians

In [79]:
anom = TANOM['TREFMXAV'].values
anom.shape

(14966, 360, 720)

In [101]:
help(np.vectorize)

Help on class vectorize in module numpy:

class vectorize(builtins.object)
 |  vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False, signature=None)
 |  
 |  vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False,
 |            signature=None)
 |  
 |  Generalized function class.
 |  
 |  Define a vectorized function which takes a nested sequence of objects or
 |  numpy arrays as inputs and returns a single numpy array or a tuple of numpy
 |  arrays. The vectorized function evaluates `pyfunc` over successive tuples
 |  of the input arrays like the python map function, except it uses the
 |  broadcasting rules of numpy.
 |  
 |  The data type of the output of `vectorized` is determined by calling
 |  the function with the first element of the input.  This can be avoided
 |  by specifying the `otypes` argument.
 |  
 |  Parameters
 |  ----------
 |  pyfunc : callable
 |      A python function or method.
 |  otypes : str or list of dtypes, optional
 |      The 

In [86]:
AMPLITUDE = np.zeros(mdl_ds['duration'].shape)
# AMPLITUDE = xr.DataArray(AMPLITUDE, dims=mdl_ds.dims, coords=mdl_ds.coords)
AMPLITUDE.shape

(1226, 360, 720)

In [100]:
%%time
# TANOM[:, i, j]  # time, lat, lon
# if we just go for it:

def get_amplitude(x, a, b):
    return np.max(x[a:b])

vamp = np.vectorize(get_amplitude)

# skip "event 0"
for i,v in enumerate(mdl_ds.events[1::], start=1):
    indx = mdl_ds['initial_index'][i, :, :].values
    dur = mdl_ds['duration'][i, :, :].values
    AMPLITUDE[i, :, :] = vamp(anom, indx, indx+dur+1)
    
    
# ilat
# jlon
#     idnum = mdl_ds['Event_ID'][:, i, j]  # events, lat, lon 
#     indx = mdl_ds['initial_index'][:, i, j]
#     dur = mdl_ds['duration'][:, i, j]
#     vals = sd.isel(time=slice(indx,indx+dur+1))
#     AMPLITUDE[idnum,]


IndexError: invalid index to scalar variable.

In [102]:
mdl_ds['initial_index'][0, :, :].values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
s = mdl_ds.isel(lat=100,lon=300)
sd = mdl_f['TREFMXAV'].isel(lat=100,lon=300)

In [99]:
mdl_ds['Event_ID'][1,:,:].values

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [23]:
events_to_check = s.events[s['initial_index'] != 0]

Really, it is only useful if we see it as anomaly.

In [51]:
sd.isel(time=slice(4, 6)).time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 2)>
array([5, 6])
Coordinates:
    lat      float64 39.75
    lon      float64 150.2
  * time     (time) object 1965-01-05 00:00:00 1965-01-06 00:00:00

In [55]:
for ev in events_to_check.values:
    idnum = s['Event_ID'].sel(events=ev)
    indx = s['initial_index'].sel(events=ev).values.item()
    dur = s['duration'].sel(events=ev).values.item()
#     print(f"Start at {indx} and slice to {indx+dur+1}")
    vals = sd.isel(time=slice(indx,indx+dur+1))
#     print(f"Max value: {vals.max().values}")

In [59]:
quants.sel(quantile=0.5)

<xarray.Dataset>
Dimensions:   (lat: 360, lon: 720, time: 365)
Coordinates:
    quantile  float64 0.5
  * lat       (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon       (lon) float64 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
Dimensions without coordinates: time
Data variables:
    TREFMXAV  (time, lat, lon) float64 ...